<a href="https://colab.research.google.com/github/krishnasaiv/chat-with-your-documents/blob/main/Chat_With_Pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps


1.   Read pdf file ( singular first )
2.   Chunk it
3.   Extract Embeddings
4.   Build an embedding index & store them in a vector database
5.   Perform Vector search with Queries & extract most relevant chunks
6.   Use the searched chunks as inputs to the model and answer the query